### Edgar Moises Hernandez Gonzalez
#### 07/02/20-08/02/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
##### Caracteristicas = RAW
##### Optimizar hiperparametros con busqueda de cuadricula
##### Validacion cruzada de 5

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01E.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)
(288, 1)


In [6]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 22000)
(576, 1)


In [7]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [8]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
x = StandardScaler().fit_transform(x)

In [10]:
#codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#convertir de 2D a 3D
x = x.reshape((n_samples, 22, 1000, 1))

print(x.shape)

(576, 22, 1000, 1)


In [0]:
def CNN_2D(n_filtros_1, n_filtros_2):
  model = Sequential()
  model.add(Conv2D(n_filtros_1, (3,3), activation='relu', input_shape=(22,1000,1)))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros_2, (3,3), activation='relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(n_classes, activation='softmax'))
    
  optimizer = Adam(lr=1e-6)
  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
clf = KerasClassifier(build_fn = CNN_2D, epochs=400, batch_size=18, verbose=0)

In [0]:
n_filtros_1 = [8, 16, 32]
#tamano_filtros_1 = [(3,3), (3,11)]
n_filtros_2 = [8, 16, 32]
#tamano_filtros_2 = [(3,3), (3,11)]
hiperparametros = dict(n_filtros_1=n_filtros_1,
                       n_filtros_2=n_filtros_2)

In [15]:
#numero de redes neuronales a evaluar
3 * 3

9

In [0]:
model = GridSearchCV(clf, hiperparametros, n_jobs=-1, cv=5)

In [17]:
model.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


GridSearchCV(cv=5, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa70ad1fa20>,
             iid='deprecated', n_jobs=-1,
             param_grid={'n_filtros_1': [8, 16, 32],
                         'n_filtros_2': [8, 16, 32]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [18]:
print(model.best_params_)
print(model.best_score_)

{'n_filtros_1': 8, 'n_filtros_2': 32}
0.37331334352493284


In [19]:
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("%0.4f (+/-%0.04f) for %r"
        % (mean, std * 2, params))

0.3384 (+/-0.1037) for {'n_filtros_1': 8, 'n_filtros_2': 8}
0.3385 (+/-0.0461) for {'n_filtros_1': 8, 'n_filtros_2': 16}
0.3733 (+/-0.0528) for {'n_filtros_1': 8, 'n_filtros_2': 32}
0.3404 (+/-0.1297) for {'n_filtros_1': 16, 'n_filtros_2': 8}
0.3612 (+/-0.0786) for {'n_filtros_1': 16, 'n_filtros_2': 16}
0.3317 (+/-0.0609) for {'n_filtros_1': 16, 'n_filtros_2': 32}
0.3715 (+/-0.0553) for {'n_filtros_1': 32, 'n_filtros_2': 8}
0.3334 (+/-0.0576) for {'n_filtros_1': 32, 'n_filtros_2': 16}
0.3508 (+/-0.0941) for {'n_filtros_1': 32, 'n_filtros_2': 32}


In [20]:
np.sort(means)

array([0.33166416, 0.33337331, 0.33842579, 0.33848575, 0.34044977,
       0.35083959, 0.36118441, 0.37145427, 0.37331334])